# Import libraries and dataframe

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [2]:
df = pd.read_csv('../project_data/complaints_processed.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

# Testing

## Make a partial dataframe for testing

In [3]:
partial_df = df[0:1000]

## Run TF-IDF on it

In [4]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(partial_df['narrative'])
feature_names = vectorizer.get_feature_names()

In [5]:
X.shape

(1000, 52129)

In [6]:
X.todense().shape

(1000, 52129)

To dense doesn't do anything.

In [7]:
lda = LatentDirichletAllocation(n_components=6,random_state=0)
lda.fit(X)

LatentDirichletAllocation(n_components=6, random_state=0)

In [8]:
for idx, topic in enumerate(lda.components_):
    print ("Topic ", idx, " ".join(feature_names[i] for i in topic.argsort()[:-10 - 1:-1]))

Topic  0 application tx credit without debt loan extend application submitted extend credit credit without
Topic  1 report per add hard add inquiry credit per hard inquiry hard inquiry bank credit
Topic  2 paid show ny paid full payment history new attachment problem reporting due see requested requested attachment
Topic  3 loan credit account payment debt attached bank see attached letter amount
Topic  4 consumer information agency block reporting section consumer reporting reporting agency identity theft
Topic  5 account credit dispute balance payment late never opened one card


It seems to work OK. Now let's do the whole dataset.

# LDA on whole dataset

## Get rid of NaN's

In [9]:
df['narrative'].isnull().sum()

10

In [10]:
df.shape

(162421, 2)

In [11]:
df.dropna().shape

(162411, 2)

In [12]:
df = df.dropna()

## TF-IDF

https://stackoverflow.com/questions/44208501/getting-topic-word-distribution-from-lda-in-scikit-learn

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(df['narrative'])
feature_names = vectorizer.get_feature_names()

In [14]:
lda = LatentDirichletAllocation(n_components=6,random_state=0)
lda.fit_transform(X)

array([[0.00946151, 0.00948227, 0.00947566, 0.95246162, 0.00965658,
        0.00946236],
       [0.01197907, 0.0119806 , 0.01197981, 0.94010069, 0.01198407,
        0.01197575],
       [0.01049076, 0.10228657, 0.01054554, 0.85568108, 0.01049898,
        0.01049708],
       ...,
       [0.08334818, 0.08333334, 0.08355956, 0.58304397, 0.08337326,
        0.08334168],
       [0.08334818, 0.08333334, 0.08355956, 0.58304397, 0.08337326,
        0.08334168],
       [0.08334818, 0.08333334, 0.08355956, 0.58304397, 0.08337326,
        0.08334168]])

In [15]:
for idx, topic in enumerate(lda.components_):
    print ("Topic ", idx, " ".join(feature_names[i] for i in topic.argsort()[:-6 - 1:-1]))

Topic  0 day demand regard credit demand demand comply filed
Topic  1 day report day item item credit credit report deleted
Topic  2 victim identity victim theft identity identity theft account
Topic  3 account credit report payment credit report information
Topic  4 claim collection couple report item account
Topic  5 consumer section consumer reporting block reporting agency agency


In [16]:
X = df.drop(['serious_accident'], axis=1)
y = df.serious_accident

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=200)

KeyError: "['serious_accident'] not found in axis"